## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

* 1054112
* 1154112
* 1254112
* 1354112

In [8]:
jan_file_path = 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet'
feb_file_path = 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-02.parquet'

In [9]:
import pandas as pd

df = pd.read_parquet(jan_file_path)

In [10]:
df.describe()

,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID
count,1154112,1154112,195845.000000,991892.000000
mean,2021-01-16 21:13:00.469540864,2021-01-16 21:32:10.502986752,139.859690,135.898030
min,2021-01-01 00:00:00,2021-01-01 00:00:24,1.000000,1.000000
25%,2021-01-09 06:32:31,2021-01-09 06:48:00,75.000000,67.000000
50%,2021-01-16 16:44:21.500000,2021-01-16 17:03:41,143.000000,132.000000
75%,2021-01-24 14:25:43,2021-01-24 14:42:23,206.000000,213.000000
max,2021-01-31 23:59:57,2021-11-01 06:03:46,265.000000,265.000000
std,NaN,NaN,74.991382,80.474902


> **Answer** `1154112`

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the average trip duration in January?

* 15.16
* 19.16
* 24.16
* 29.16

In [11]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [15]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime

In [20]:
df['duration'] = df['duration'].dt.total_seconds()/60

In [23]:
round(df['duration'].mean(), 2)

19.17

> **Answer** `19.16`

## Data preparation

Check the distribution of the duration variable. There are some outliers. 

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop? 

In [25]:
df.shape

(1154112, 8)

In [33]:
# df = df[df.duration]
df = df[(df.duration >= 1) & (df.duration < 60)]

In [34]:
df.shape

(1107511, 8)

In [35]:
1154112 - 1107511

46601

> `46601` records drop

## Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs. 

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

* 53%
* 63%
* 73%
* 83%

In [37]:
df.isnull().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               926063
DOlocationID               146976
SR_Flag                   1107511
Affiliated_base_number        548
duration                        0
dtype: int64

In [40]:
df.fillna(value = -1, inplace=True)

In [42]:
926063 * 100/ 1107511

83.61659613313095

> **Answer** `83%`

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

* 2
* 152
* 352
* 525
* 725

In [48]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,-1,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,-1,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,-1,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,-1,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,-1,B00037,9.050000


In [56]:
df['PUlocationID'] = df['PUlocationID'].astype(str)

In [58]:
df['DOlocationID'] = df['DOlocationID'].astype(str)

In [59]:
data_dicts = df[['PUlocationID', 'DOlocationID']].to_dict(orient='records')

In [60]:
from sklearn.feature_extraction import DictVectorizer

In [61]:
dv = DictVectorizer(sparse=False)

In [62]:
X_train = dv.fit_transform(data_dicts)

In [63]:
X_train.shape

(1107511, 525)

> **Answer** `525`

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 5.52
* 10.52
* 15.52
* 20.52

In [64]:
y_train = df['duration']

In [65]:
y_train

0          17.000000
1          17.000000
3           8.283333
4          15.216667
5           9.050000
             ...    
1154107     8.750000
1154108    57.600000
1154109    16.200000
1154110    19.433333
1154111    36.000000
Name: duration, Length: 1107511, dtype: float64

In [67]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(y, y_pred):
    mse = mean_squared_error(y_train, y_pred)
    rmse_value = mse ** 0.5
    return rmse_value

rmse(y_train, y_pred)

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021). 

What's the RMSE on validation?

* 6.01
* 11.01
* 16.01
* 21.01

In [ ]:
df2 = pd.read_parquet(feb_file_path)
df2['duration'] = df2.dropOff_datetime - df2.pickup_datetime
df2['duration'] = df2['duration'].dt.total_seconds()/60

In [ ]:
df2 = df2[(df2.duration >= 1) & (df2.duration < 60)]

In [ ]:
df2.fillna(value=-1, inplace=True)
df2['PUlocationID'] = df2['PUlocationID'].astype(str)
df2['DOlocationID'] = df2['DOlocationID'].astype(str)
val_dicts = df2[['PUlocationID', 'DOlocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df2.duration
y_pred = model.predict(X_val)
rmse(y_val, y_pred)